# Serve Mistral 7B Instruct v0.2 using Triton Inference Server with vLLM on AWS Neuron

This notebook shows how to serve [Mistral-7B-Instruct-v0.2](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2) model using [Triton Inference Server](https://github.com/triton-inference-server) with [vLLM backend](https://github.com/triton-inference-server/vllm_backend/tree/main) on [AWS Neuron](https://awsdocs-neuron.readthedocs-hosted.com/en/latest/index.html).

## Setup and Imports

In [ ]:
! pip install kubernetes

In [ ]:
import os
import subprocess
import sys

# Set working directory
os.chdir(os.path.expanduser('~/amazon-eks-machine-learning-with-terraform-and-kubeflow'))
print(f"Working directory: {os.getcwd()}")

# Get the src directory
src_dir = os.path.join(os.getcwd(), "src")
sys.path.insert(0, src_dir)

from k8s.utils import (
    wait_for_helm_release_pods,
    wait_for_triton_server,
    find_matching_helm_services
)

# Get notebook directory
notebook_dir = os.path.join(os.getcwd(), 'examples', 'inference', 'triton-inference-server', 
                            'vllm_backend', 'mistral-7b-instruct-v02-neuron')
print(f"Notebook directory: {notebook_dir}")

# initialize key variables
release_name = 'triton-server-mistral-7b-instruct-v02-vllm-nx'
namespace = 'kubeflow-user-example-com'
hf_model_id = 'mistralai/Mistral-7B-Instruct-v0.2'

## Step 1: Download Hugging Face Mistral 7B Instruct v0.2 Model Weights

**Note:** Set your Hugging Face token below before running cell.

In [ ]:
# Replace with your actual Hugging Face token
HF_TOKEN = None
assert HF_TOKEN, "Please set HF_TOKEN"

cmd = [
    'helm', 'install', '--debug', release_name,
    'charts/machine-learning/model-prep/hf-snapshot',
    '--set-json', f'env=[{{"name":"HF_MODEL_ID","value":"{hf_model_id}"}},{{"name":"HF_TOKEN","value":"{HF_TOKEN}"}}]',
    '-n', namespace
]

result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print("STDERR:", result.stderr)

In [ ]:
# Wait for model download to complete
wait_for_helm_release_pods(release_name, namespace)

In [ ]:
# Uninstall the model download job
cmd = ['helm', 'uninstall', release_name, '-n', namespace]
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print("STDERR:", result.stderr)

## Step 2: Launch Triton Server

In [ ]:
cmd = [
    'helm', 'install', '--debug', release_name,
    'charts/machine-learning/serving/triton-inference-server',
    '-f', f'{notebook_dir}/triton_server.yaml',
    '-n', namespace
]

result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print("STDERR:", result.stderr)

In [ ]:
wait_for_triton_server(release_name, namespace)

## Step 3: Check Service Status

In [ ]:
services = find_matching_helm_services(release_name, namespace)
for service in services:
    print(f"Service {service.metadata.name} is available.")
    print(f"Service type: {service.spec.type}")
    print(f"Service ports: {service.spec.ports} ")

## Step 4: Stop Service

Run this cell when you want to stop the Triton Inference Server service.

In [ ]:

cmd = ['helm', 'uninstall', release_name, '-n', namespace]
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print("STDERR:", result.stderr)